
# Staffing Optimization Notebook

This notebook demonstrates how to determine the **best staffing configuration** and **optimal staff assignment** using **linear programming**.  

We will:
1. Define a function to compute the **optimal number of staff per role** based on workload (orders forecast).
2. Define another function that assigns **actual staff members to roles** using **linear optimization**.
3. Showcase how both functions work together with an example.

needed libraries : !pip install pandas pulp



In [ ]:

# Importing necessary libraries
import pandas as pd
import math
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus



### Step 1 – Staffing Configuration
The function `get_staffing_config` defines a **rule-based staffing approach**:  

- If the **forecasted orders** are within a normal workload → assign **1 person per role**.  
- If the workload is **supercharged ** → assign **2 people per role**.  

This is a **simplified deterministic rule** that acts as the input for the next optimization step.


In [ ]:
import pandas as pd
import math

def get_staffing_config(orders_forecast: int) -> dict:
    """
    Returns staffing configuration based on forecasted orders and rules.
    Uses dynamic scaling: ~1 staff per 50 orders, capped by role max.
    """
    rules = {
        "dough": (1, 2),
        "topping": (1, 2),
        "cashier": (1, 2),
        "waiter": (1, 3),
        "delivery": (1, 2),
        "packaging": (1, 2),
        "bar": (1, 2)
    }

    # scale grows with demand: 50 orders = 1 "load unit"
    scale = math.ceil(orders_forecast / 50) if orders_forecast > 0 else 1

    staffing = {}
    for role, (regular, max_staff) in rules.items():
        staffing[role] = min(regular + (scale - 1), max_staff)

    return staffing


# Example shift forecasts (sum of hourly predictions per shift)
forecasted_orders = {
    "8am-4pm": 48,    # low demand
    "4pm-1am": 120,   # higher demand
}

# Build staffing schedule
schedule = {}
for shift, orders in forecasted_orders.items():
    schedule[shift] = get_staffing_config(orders)

# Convert to DataFrame for nice tabular view
df_staffing = pd.DataFrame(schedule).T
print(df_staffing)



### Step 2 – Linear Programming for Staff Assignment
We now use **Linear Programming (LP)** to decide which staff should work in each role.  

- **Decision Variables:**  
  `x[s, r] = 1 if staff s is assigned to role r, 0 otherwise.`  

- **Objective Function:**  
  Minimize the total wages + a strong penalty for temporary assigments (trying the minimize the wages while giving huge penalty of temporary assignments will result in choosing the best assingments ) 

- **Constraints:**  
  1. all needs are fullfilled :  
     sum(assignments for role) + TEMP = needs[role]
 
  2. no one works two roles in a shift:  
    sum(x[staff, r]) <= 1

  3. If delivery staff exist, they must cover delivery needs (no TEMP allowed).


The solver finds a feasible assignment that satisfies all constraints.


In [ ]:
import pulp

staff_pool = [
    {"name": "Ahmed",   "wage": 2500, "skills": ["cashier", "packaging"]},
    {"name": "Youssef", "wage": 3000, "skills": ["dough", "bar"]},
    {"name": "Khaled",  "wage": 2700, "skills": ["topping", "waiter"]},
    {"name": "Rania",   "wage": 2400, "skills": ["waiter", "packaging"]},
    {"name": "Fatima",  "wage": 3200, "skills": ["bar", "dough", "topping"]},
    {"name": "Samir",   "wage": 2600, "skills": ["cashier", "waiter", "packaging"]},
    {"name": "Houda",   "wage": 3100, "skills": ["dough", "topping", "packaging"]},
    {"name": "Zineb",   "wage": 2900, "skills": ["waiter", "cashier", "bar"]},
    {"name": "Karim",   "wage": 2500, "skills": ["topping", "dough"]},
    {"name": "Leila",   "wage": 2600, "skills": ["cashier", "packaging"]},
    {"name": "Omar",    "wage": 2400, "skills": ["packaging", "bar"]},
    {"name": "Nour",    "wage": 2500, "skills": ["cashier", "topping"]},
    {"name": "Salima",  "wage": 2600, "skills": ["waiter", "packaging"]},
    {"name": "Hassan",  "wage": 2700, "skills": ["bar", "topping"]},
    {"name": "Amel",    "wage": 2550, "skills": ["topping", "cashier"]},
    {"name": "Djamila", "wage": 2800, "skills": ["delivery"]},
    {"name": "Imane",   "wage": 2700, "skills": ["delivery"]},
    {"name": "Walid",   "wage": 2600, "skills": ["delivery"]},
    {"name": "Salem",   "wage": 2500, "skills": ["delivery"]},
    {"name": "Nadia",   "wage": 2550, "skills": ["delivery"]},
]







def optimize_staff_assignment(staff_pool, needs, forbidden=[]):
    """
    Optimizes staff assignment given pool, staffing needs, and forbidden staff (e.g. night→morning rule).
    Delivery cannot be TEMP-hired. Guarantees coverage of all roles.
    """
    roles = list(needs.keys())
    prob = pulp.LpProblem("StaffAssignment", pulp.LpMinimize)

    # Decision variables

    x = {}
    for s in staff_pool:
        if s["name"] in forbidden:
            continue
        for r in roles:
            if r in s["skills"]:
                x[(s["name"], r)] = pulp.LpVariable(f"{s['name']}_{r}", 0, 1, cat="Binary")

    # TEMP hire variables (delivery forbidden)
    for r in roles:
        if r == "delivery":
            x[("TEMP", r)] = pulp.LpVariable(f"TEMP_{r}", 0, 0, cat="Integer")  # forbid TEMP
        else:
            x[("TEMP", r)] = pulp.LpVariable(f"TEMP_{r}", 0, needs[r], cat="Integer")


    # Objective: wages + strong TEMP penalty
 
    prob += (
        pulp.lpSum([
            (s["wage"] if r != "delivery" else s["wage"] * 0.8) * x[(s["name"], r)]
            for s in staff_pool for r in roles
            if (s["name"], r) in x
        ])
        + pulp.lpSum([20000 * x[("TEMP", r)] for r in roles])  # TEMP very expensive
    )

    # Constraints

    # Coverage: meet exact needs
    for r in roles:
        prob += (
            pulp.lpSum([x[(s["name"], r)] for s in staff_pool if (s["name"], r) in x])
            + x[("TEMP", r)]
            == needs[r]
        )

    # Each staff at most one role
    for s in staff_pool:
        if s["name"] in forbidden:
            continue
        prob += pulp.lpSum([x[(s["name"], r)] for r in roles if (s["name"], r) in x]) <= 1

    # Delivery guarantee (only if staff exist for it)
    if "delivery" in needs:
        delivery_staff = [s for s in staff_pool if "delivery" in s["skills"]]
        if delivery_staff:  # only enforce if we have delivery-capable staff
            prob += (
                pulp.lpSum([
                    x[(s["name"], "delivery")]
                    for s in delivery_staff
                    if (s["name"], "delivery") in x
                ]) >= needs["delivery"]
            )

    # Limit TEMP hires to 1 per role (except delivery which is 0 already)
    for r in roles:
        if r != "delivery":
            prob += x[("TEMP", r)] <= 1

    # Solve
    prob.solve(pulp.PULP_CBC_CMD(msg=0))

    # Collect assignment
    assignment = {r: [] for r in roles}
    for (s_name, r), var in x.items():
        if pulp.value(var) > 0.5:
            if s_name == "TEMP":
                assignment[r].append("TEMP-HIRE")
            else:
                assignment[r].append(s_name)

    return assignment



# Example Run 

orders_forecast = 120   # e.g. high demand case
needs = get_staffing_config(orders_forecast)  # <-- call your version

# Example: forbid Ahmed (pretend he worked night shift yesterday)
optimal_config = optimize_staff_assignment(staff_pool, needs, forbidden=["Ahmed"])

print("Optimal Assignment:")
for role, staff in optimal_config.items():
    print(f" {role}: {', '.join(staff) if staff else 'None'}")



### Step 3 – Example Run
- The **staffing configuration** is first computed based on the workload forecast.  
- Then, the **linear programming solver** assigns real people to roles while respecting the required constraints.  

This ensures the restaurant has the right number of staff per role and avoids over-assigning anyone.


In [ ]:
import pandas as pd
from datetime import timedelta


# 1. Create realistic hourly forecast (5 days × 24h)

dates = pd.date_range("2025-09-26", periods=5, freq="D")
hours = list(range(24))

data = []
for d in dates:
    for h in hours:
        # Simulate realistic daily demand pattern
        if 7 <= h < 10:   # breakfast bump
            orders = 40
        elif 12 <= h < 14:  # lunch rush
            orders = 80
        elif 19 <= h < 22:  # dinner rush
            orders = 150
        elif 23 <= h or h < 3:  # late night very low
            orders = 10
        else:
            orders = 25
        data.append({"date": d, "hour": h, "orders": orders})

df_forecast = pd.DataFrame(data)

# 2. Build schedule using existing functions
def build_schedule(df_forecast, staff_pool):
    schedule = []
    prev_night_staff = set()  # to avoid night → morning

    for d in sorted(df_forecast["date"].dt.date.unique()):
        for shift_name, hours in [("Day", range(8,16)), ("Night", list(range(16,24))+[0])]:
            # Sum orders for the shift
            shift_orders = df_forecast[
                (df_forecast["date"].dt.date == d) &
                (df_forecast["hour"].isin(hours))
            ]["orders"].sum()

            # Get staffing config
            needs = get_staffing_config(shift_orders)

            # Apply continuity rule
            forbidden = prev_night_staff if shift_name == "Day" else []

            # Optimize staff assignment (calls your existing function)
            assignment = optimize_staff_assignment(staff_pool, needs, forbidden)

            # Save result
            schedule.append({
                "date": d,
                "shift": shift_name,
                "orders_forecast": shift_orders,
                "assignment": assignment
            })

            # Track who worked night to forbid them next morning
            prev_night_staff = set(sum(assignment.values(), [])) if shift_name == "Night" else prev_night_staff

    return schedule

# 3. Run and display
schedule = build_schedule(df_forecast, staff_pool)

for s in schedule:
    print(f"\n {s['date']} - {s['shift']} (Forecast Orders: {s['orders_forecast']})")
    for role, staff in s["assignment"].items():
        print(f"  {role}: {', '.join(staff) if staff else 'None'}")
